In [ ]:
# Healthcare Python Streaming Application Demo

This Python notebooks shows how you can create an app that ingests and analyzes streaming data from a feed, and then visualizes the data in the notebook.  You submit this ap to be built in the Streaming Analyics service instance in Bluemix.

The app is a Python Topology app written with the Streams Python API. The app subscribes to the topic containing the patient data, performs analysis on the waveforms and send all of the data, including the results of the analysis, to the Streams view server.

Submitting the Python application from the notebook allows for connecting to the Streams view server in order to retrieve the data. Once the data has been retrieved, it can be analyzed, manipulated or visualized like any other data accessed from a notebook. In the case of this demo, waveform graphs and numerical widgets display the healthcare data of the patient.

The following diagram outlines the architecture of the demo.

![Demo Architecture](https://github.com/IBMStreams/streamsx.health/blob/develop/samples/HealthcareJupyterDemo/images/architecture_diagram.jpg?raw=true)

## Table of contents

This notebook is divided into parts:

1. [Part 1: Setup](#setup)
1. [Part 2: Create a data feed](#createfeed)
1. [Part 3: Build a streaming app](#buildapp)
1. [Part 4: Visualization](#visualization)

<a id="setup"></a>
## Part 1: Setup

Complete the following steps to set up your DSX environment.

### 1.1 Install the Python modules

Run the following cell to install the latest software package.

In [ ]:
!pip install --user --upgrade streamsx
!pip install --user --upgrade "git+https://github.com/IBMStreams/streamsx.health.git#egg=healthdemo&subdirectory=samples/HealthcareJupyterDemo/package"

### 1.2 Start a Streaming Analytics service instance

If you have a Streaming Analytics instance in IBM Bluemix, make sure that it's up and running.

If you don't have a Streaming Analytics instance, create one with these steps:

1. Go to the [Bluemix web portal](https://www.ibm.com/cloud-computing/bluemix/) and log in (or sign up for a free Bluemix account).
1. Click **Catalog**, browse for the **Streaming Analytics service** and then click on it. 
1. Follow the instructions on the Streaming Analytics catalog page to set up your service instance, and then click **Create**.
The service dashboard opens and your service instance starts automatically.

### 1.3 Set up access to the service instance

You must provide the information that your streaming app needs to access the service instance.

Start by running the next cell and then entering the name of your service instance.

In [ ]:
service_name=input('Streaming Analytics service name:')

Now run the following cell, and then enter your list of service credentials.

*Tip:* To copy and paste your service credentials, go to your service dashboard, click *Service Credentials* and then *View Credentials*

In [ ]:
import json
import getpass
from streamsx.topology.context import ConfigParams
credentials=getpass.getpass('Streaming Analytics credentials:')

vs={'streaming-analytics': [{'name': service_name, 'credentials': json.loads(credentials)}]}
config={ConfigParams.VCAP_SERVICES: vs, ConfigParams.SERVICE_NAME: service_name}

<a id="createfeed"></a>
## Part 2: Create a data feed

We'll use a pre-compiled IBM Streams application to simulate patient data, and publish the data to a topic. A service is a small IBM Streams application. The PhysionetIngestServiceMulti microservice retrieves patient waveform and vital data from a Physionet database (https://www.physionet.org/) using three different sets of data. The patient data is submitted to the ingest-physionet topic so that the data feed can be consumed by the apps you're going to build later in this notebook.

### 2.1 Submit the precompiled Physionet microservice

1. Download the PhysionetIngestServiceMulti.sab file to your local machine from https://github.com/IBMStreams/streamsx.health/releases.
1. Go to the Streaming Analytics service instance and submit the SAB file by using the Streams application development console.
1. When prompted, set the number of patients to 3 or more.

In [ ]:
print ("Don't forget to submit patient ingest microservice manually. (Set num.patients >= 3)")

<a id="buildapp"></a>
## Part 3: Build an streaming app

Now you're ready to create the **HealthcareDemo** Python streaming application, and submit it to the Streaming Analytics service instance, where the app will be remotely built and deployed.

### 3.1 Healthcare Patient Python Topology Application

This cell contains source code for the Python Topology application. This is a Streaming Python application that ingests the patient data from the *ingest-physionet* topic, and performs analysis on the patient data to calculate vital data for all patients and then sends the data to the Streams view server.

In [ ]:
from streamsx.topology.topology import Topology, schema
from streamsx.topology.context import submit
from healthdemo.patientmonitoring_functions import streaming_rpeak
from healthdemo.healthcare_functions import GenTimestamp, aggregate
from healthdemo.windows import SlidingWindow

topo = Topology('HealthcareDemo')

## The ingest-physionet provides data at a rate of 125 tuples/sec
sample_rate = 125

## Subscribe to the topic, gather tuples, aggregate the data to create a 1 tuple/sec stream
patients_data = topo.subscribe('ingest-physionet', schema.CommonSchema.Json) \
                    .transform(GenTimestamp(sample_rate)) \
                    .transform(SlidingWindow(length=sample_rate, trigger=sample_rate-1)) \
                    .transform(aggregate)

## Calculate RPeak and RR delta
patients_data = streaming_rpeak(patients_data, sample_rate, data_label='ECG Lead II')

## Create view for patient vital data
patients_vital = patients_data.view(name='patients_vital')

rc = submit('ANALYTICS_SERVICE', topo, config)
print ("DONE")

<a id="visualization"></a>
## Part 4: Visualization

### 4.1 Setup graphs for plotting patient vitals

This cell initializes all of the graphs that will be used as well as creates the background job that access the view data.

The view data is continuously retrieved from the Streams view server in a background job. Each graph object receives a copy of the data. The graph objects extracts and stores the data that is relevant for that particular graph. Each time a call to ```update()``` is made on a graph object, the next data point is retrieved and displayed. Each graph object maintains an internal queue so that each time a call to ```update()``` is made, the next element in the queue is retrieved and removed. 

In [ ]:
import time
from healthdemo.medgraphs import ECGGraph, PoincareGraph, NumericText, ABPNumericText

## load BokehJS visualization library (must be loaded in a separate cell)
from bokeh.io import output_notebook, push_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
%autosave 0
%reload_ext autoreload
%aimport healthdemo.utils
%aimport healthdemo.medgraphs
%autoreload 1

## Initialize graph for a patient
patientId = 'patient-1'
continue_data_collection = True

graph = {
    'ecg_leadII_graph': ECGGraph(signal_label='ECG Lead II', title='ECG Lead II', plot_width=600, min_range=-0.5, max_range=2.0),
    'leadII_poincare': PoincareGraph(signal_label='Poincare - ECG Lead II', title='Poincare - ECG Lead II'),
    'ecg_leadV_graph': ECGGraph(signal_label='ECG Lead V', title='ECG Lead V', plot_width=600),
    'resp_graph': ECGGraph(signal_label='Resp', title='Resp', min_range=-1, max_range=3, plot_width=600),
    'pleth_graph': ECGGraph(signal_label='Pleth', title='Pleth', min_range=0, max_range=5, plot_width=600),
    'hr_numeric': NumericText(signal_label='HR', title='HR', color='#7cc7ff'),
    'pulse_numeric': NumericText(signal_label='PULSE', title='PULSE', color='#e71d32'),
    'spo2_numeric': NumericText(signal_label='SpO2', title='SpO2', color='#8cd211'),
    'abp_numeric': ABPNumericText(abp_sys_label='ABP Systolic', abp_dia_label='ABP Diastolic', title='ABP', color='#fdd600')            
}

## retrieve data from Streams view in a background job
def data_collector(view, g):
    queue = view.start_data_fetch()
    while continue_data_collection:
        tup = queue.get()
        if patientId == get_patient_id(tup):
            for graphtype in g:
                g[graphtype].add(tup)
    view.stop_data_fetch()
            
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()
jobs.new(data_collector, patients_vital, graph)

### 4.2 Display the graphs

This cell is responsible for laying out and displaying the graphs. There is a loop that continuously calls the ```update()``` method on each of the graphs for 60 seconds. After each graph has been updated, a call to ```push_notebook()``` is made, which causes the notebook to update the graphics.

In [ ]:
import time
from bokeh.io import show
from bokeh.layouts import column, row, widgetbox

## display graphs for a patient
show(
    row(
        column(
            graph['ecg_leadII_graph'].get_figure(), 
            graph['ecg_leadV_graph'].get_figure(), 
            graph['resp_graph'].get_figure(),
            graph['pleth_graph'].get_figure()
        ), 
        column(
            graph['leadII_poincare'].get_figure(),
            widgetbox(graph['hr_numeric'].get_figure()),
            widgetbox(graph['pulse_numeric'].get_figure()),
            widgetbox(graph['spo2_numeric'].get_figure()),
            widgetbox(graph['abp_numeric'].get_figure())
        )
    ),
    
    # If using bokeh > 0.12.2, uncomment the following statement
    #notebook_handle=True
)

## Timeout(in seconds) before stopping the graph
timeout = 60
endtime = time.time() + timeout

cnt = 0
while time.time() < endtime:
    ## update graphs
    for graphtype in graph:
        graph[graphtype].update()

    ## update notebook 
    cnt += 1
    if cnt % 5 == 0:
        push_notebook() ## refresh the graphs
        cnt = 0
    time.sleep(0.008)
    
# Stop data collection running in background thread
continue_data_collection = False